In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file 

In [ ]:
train_df=pd.read_csv('csv_files/train.csv')
test_df=pd.read_csv('csv_files/test.csv')

In [ ]:
train_df.describe()

In [ ]:
train_df.head(10)

In [ ]:
import spacy

In [ ]:
pd.crosstab(index=train_df['label'],columns='count')

In [ ]:
import seaborn as sns
sns.countplot(x='label',data=train_df)

Empty model with Spacy , use TextCategorizer within model

In [ ]:
nlp=spacy.blank('en')
classifier = nlp.create_pipe('textcat',config={'exclusive_classes':True,'architecture' : 'bow'})
nlp.add_pipe(classifier)
classifier.add_label('RACIST/SEXIST')
classifier.add_label('NON_RACIST/SEXIST')

1

Convert tweet text data to the input form for TextCategorizer. Set separate labels for the classes 'Racist/sexist' and 'Non-racist/sexist'

In [ ]:
train_data=train_df['tweet'].values
train_label=[{'cats':{'RACIST/SEXIST':label==1,
                    'NON_RACIST/SEXIST':label==0}} for label in train_df['label']]
train_list=list(zip(train_data,train_label))

In [ ]:
from spacy.util import minibatch
import random
random.seed(1)
spacy.util.fix_random_seed(1)
optimizer=nlp.begin_training()
losses={}


Training the model 

In [ ]:
for epoch in range(10):
    random.shuffle(train_list)
    batches=minibatch(train_list,size=24)
    for batch in batches:
        text,labels=zip(*batch)
        nlp.update(text,labels,sgd=optimizer,losses=losses)
        #print(losses,batch,epoch)

In [ ]:
test_df.head(10)
test_df['tweet']

In [ ]:
test=[nlp.tokenizer(text) for text in test_df['tweet']]


Predict results on test data

In [ ]:
classifier=nlp.get_pipe('textcat')
scores,_=classifier.predict(test)
print(scores)

[[1.8355710e-02 9.8164427e-01]
 [6.8968111e-01 3.1031892e-01]
 [1.5408030e-05 9.9998462e-01]
 ...
 [3.1029875e-03 9.9689698e-01]
 [4.3769032e-06 9.9999559e-01]
 [2.0392453e-04 9.9979609e-01]]


In [ ]:
predict_label=scores.argmax(axis=1)
count=0
for pred in predict_label:
  print(test_df['tweet'][count],'racist/sexist'if pred==0 else "non_racist/sexist")
  count+=1